-   products: list of product names (e.g., ['d', 't', 'c'])
-   scenarios: list of scenario names (e.g., ['l', 'm', 'h'])
-   cost_x: dict, cost per unit of each resource {r: cost}
-   profit_y: dict, profit per unit of each product {p: profit}
-   usage: nested dict, usage[r][p] = units of resource r needed per unit of product p
-   p: dict, probability of each scenario {s: probability}
-   demand_ub: nested dict, demand_ub[s][p] = upper bound on product p in scenario s


In [1]:
from gurobipy import Model, GRB

In [2]:
def two_stage_model(resources, products, scenarios, cost_x, profit_y, usage, p, demand_ub):
    
    model = Model("general_two_stage_stochastic")
    
    x = model.addVars(resources, lb=0.0, name="x") 

    y = model.addVars(products, scenarios, lb=0.0, name="y") 

    first_stage_cost = sum(cost_x[r] * x[r] for r in resources)
    recourse_profit = sum(
        p[s] * sum(profit_y[i] * y[i, s] for i in products)
        for s in scenarios
    )
    model.setObjective(first_stage_cost - recourse_profit, GRB.MINIMIZE)

    for s in scenarios:
        for r in resources:
            model.addConstr(
                -x[r] + sum(usage[r][i] * y[i, s] for i in products) <= 0,
                name=f"capacity_{r}_{s}"
            )

    for s in scenarios:
        for i in products:
            model.addConstr(
                y[i, s] <= demand_ub[s][i],
                name=f"demand_ub_{i}_{s}"
            )
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print("First‐stage x*:")
        for r in ['b','f','c']:
            print(f" x_{r} = {x[r].X:.2f}")
        print("\nRecourse y* (per scenario):")
        for w in scenarios:
            print(f" Scenario {w}: ",
                ", ".join(f"{i}={y[i,w].X:.0f}" for i in ['d','t','c']))
    else:
        print("No optimal solution found.")


In [ ]:
resources = ['b', 'f', 'c']
products = ['d', 't', 'c']
scenarios = ['l', 'm', 'h']

cost_x = {'b': 2, 'f': 4, 'c': 5.2}
profit_y = {'d': 60, 't': 40, 'c': 10}

usage = {
    'b': {'d': 8, 't': 6, 'c': 1},
    'f': {'d': 4, 't': 2, 'c': 1.5},
    'c': {'d': 2, 't': 1.5, 'c': 0.5}
}

p = {'l': 0.3, 'm': 0.4, 'h': 0.3}

demand_ub = {
    'l': {'d': 50, 't': 20, 'c': 200},
    'm': {'d': 150, 't': 110, 'c': 225},
    'h': {'d': 250, 't': 250, 'c': 500}
}

two_stage_model(resources, products, scenarios, cost_x, profit_y, usage, p, demand_ub)


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 18 rows, 12 columns and 45 nonzeros
Model fingerprint: 0xfa9bc9a3
Coefficient statistics:
  Matrix range     [5e-01, 8e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+02]
Presolve removed 12 rows and 0 columns
Presolve time: 0.01s
Presolved: 6 rows, 12 columns, 42 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -9.0820000e+03   5.193750e+02   0.000000e+00      0s
       8   -1.7300000e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.730000000e+03
First‐stage x*:
 x_b = 1300.00
 x_f = 540.00
 x